In [1]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score ,GridSearchCV
import joblib
import json
import os

In [5]:
weather_df = pd.read_csv(f"../資料前處理以及第三方資料/weather_sun_combine.csv")
weather_df["日期"] = pd.to_datetime(weather_df["日期"])
weather_df["location"] = weather_df["location"].astype(str).str.zfill(2)
weather_df["時間"] = weather_df["時間"].astype(str).str.zfill(2)
weather_df["時段"] = weather_df["時段"].astype(str).str.zfill(2)

In [4]:
model_folder_name = "train_weatherdata_all"
#將不同模型load近來並放在字典裡
model_dict = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    model_file_path = os.path.join(f"./model_each_location/{model_folder_name}/{location}_weather.pkl")
    model_dict[location] = joblib.load(model_file_path)

In [10]:
#將檔案讀進來  新增日期時間位置等欄位以便我們找資料對應
DataName = "./36_TestSet_SubmissionTemplate/upload(no answer).csv"
ans = pd.read_csv(DataName, encoding='utf-8')
ans["序號"] = ans["序號"].astype(str)
ans["日期"] = pd.to_datetime(ans["序號"].str.slice(0,8))
ans["時間"] =ans["序號"].str.slice(8,10)
ans["時段"] =ans["序號"].str.slice(10,12)
ans["位置"] =ans["序號"].str.slice(12,14)
ans["模型順位"] = None
ans["使用模型MAE"] = None
select_cols = [ "TX01","GR01","RH01","UV01",
                "AirTemperature","RelativeHumidity","SunshineDuration","UVIndex",
                "太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"
                ]
for i in range(len(ans)):
    date,time,timecount,ans_location = ans.loc[i,"日期"],ans.loc[i,"時間"],ans.loc[i,"時段"],ans.loc[i,"位置"]  #找出答案該個row的時期時間時段以及位置
    search_df = weather_df.loc[(weather_df["日期"]==date) & (weather_df["時間"]==time) & (weather_df["時段"]==timecount) & (weather_df["location"]==ans_location),]

    if len(search_df)==1:  #如果有找資料 就用這個模型predict
        X_test = search_df.loc[:,select_cols]
        model = model_dict[ans_location]
        predictions = model.predict(X_test)
        ans.loc[i,"答案"] = predictions[0]
    if i %100==0:
        print(i)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500


In [11]:
ans.loc[ans["答案"].isna(),]

,序號,答案,日期,時間,時段,位置,模型順位,使用模型MAE


In [ ]:
ans.loc[:,["序號","答案"]].to_csv("predict_use_weather_data.csv",index=False)